In [75]:
import requests
from datasets import load_dataset, get_dataset_split_names
import collections
import re

In [6]:
dataset = load_dataset("microsoft/ms_marco", "v1.1", cache_dir="data/marco")

Generating validation split:   0%|          | 0/10047 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/82326 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/9650 [00:00<?, ? examples/s]

In [62]:
train_data = dataset['train']

In [68]:
train_data[1]

{'answers': ['Yes'],
 'passages': {'is_selected': [0, 1, 0, 0, 0, 0, 0],
  'passage_text': ['In his younger years, Ronald Reagan was a member of the Democratic Party and campaigned for Democratic candidates; however, his views grew more conservative over time, and in the early 1960s he officially became a Republican. In November 1984, Ronald Reagan was reelected in a landslide, defeating Walter Mondale and his running mate Geraldine Ferraro (1935-), the first female vice-presidential candidate from a major U.S. political party.',
   "From Wikipedia, the free encyclopedia. A Reagan Democrat is a traditionally Democratic voter in the United States, especially a white working-class Northerner, who defected from their party to support Republican President Ronald Reagan in either or both the 1980 and 1984 elections. During the 1980 election a dramatic number of voters in the U.S., disillusioned with the economic 'malaise' of the 1970s and the presidency of Jimmy Carter (even more than, four

In [ ]:
idx = 42

query = train_data[idx].get("query", "")
answers = train_data[idx].get("answers", [])

full_text = query + "? " + " ".join(answers)
full_text

'what animal is a possum? Foxes, cats and birds. A possum (plural form: possums) is any of about 70 small-to medium-sized arboreal marsupial species native to Australia, New Guinea, and Sulawesi (and introduced to New Zealand and China).'

In [84]:
corpus = []
for sample in train_data.select(range(30)):
    query = sample.get("query", "")
    answers = sample.get("answers", [])
    full_text = query + " " + " ".join(answers)
    corpus.append(full_text.strip())

In [86]:
def preprocess(text: str) -> list[str]:
    if not isinstance(text, str):
        return ""
    
    # Normalise some symbols
    text = text.lower()
    text = re.sub(r"&", "and", text)
    text = re.sub(r"%", "percent", text)

    # Replace specific punctuation with tokens
    text = text.replace("<", "").replace(">", "") # do this here so that </> can be included in tokens.
    # text = text.replace("-", " ")
    text = text.replace("-", ' <HYPHEN> ')
    text = text.replace('.',  ' <PERIOD> ')
    text = text.replace(',',  ' <COMMA> ')
    text = text.replace(';',  ' <SEMICOLON> ')
    text = text.replace('!',  ' <EXCLAMATION_MARK> ')
    text = text.replace('?',  ' <QUESTION_MARK> ')

    # Remove any other punctuation.
    text = re.sub(r'[^\w\s<>]', '', text)
    words = text.split()
    stats = collections.Counter(words)
    words = [word for word in words if stats[word] > 0]
    return words

def process_and_concat(strings, delimiter="<DELIMIT>"):
    return f" {delimiter} ".join(" ".join(preprocess(s)) for s in strings)

In [87]:
preprocess(corpus[0])

['what',
 'is',
 'rba',
 'results',
 '<HYPHEN>',
 'based',
 'accountability',
 'is',
 'a',
 'disciplined',
 'way',
 'of',
 'thinking',
 'and',
 'taking',
 'action',
 'that',
 'communities',
 'can',
 'use',
 'to',
 'improve',
 'the',
 'lives',
 'of',
 'children',
 '<COMMA>',
 'youth',
 '<COMMA>',
 'families',
 '<COMMA>',
 'adults',
 'and',
 'the',
 'community',
 'as',
 'a',
 'whole',
 '<PERIOD>']

In [ ]:
corpus = process_and_concat(corpus)
corpus